In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import logging
from transformers import pipeline
from transformers import BertTokenizer, BertModel
from tqdm import tqdm
from PipelineCacheWrapper.PipelineCacheWrapper import PipelineCacheWrapper

In [2]:
from TRExData.LamaTRExData import LamaTRExData
from TRExData.AbstractTRExData import AbstractTRExData
from TRExData.HardTRExData import HardKnownTRExData
from SentenceTypologyQueryResults import SentenceTypologyQueryResults
from relation_templates.templates import relations, nominalized_relations, get_templates, get_relation_meta, relations, relation_names, get_relation_cardinality, get_relations_with_last_digit
from ModelHelpers.data_helpers import create_known_filter_getter, create_hard_filter_getter

In [3]:
from TypologyQuerier import TypologyQuerier 
from ResultData.ResultPersister import ResultPersister

In [4]:
#model='bert-base-cased'
#model='bert-base-uncased'
#model='bert-large-cased'
#model='bert-large-uncased'
#model='bert-base-multilingual-cased'
#model='bert-base-multilingual-uncased'
#model='roberta-base'
#model='roberta-large'
#model = "google/electra-large-generator"
model = "google/electra-base-generator"

In [5]:
#data_source = "abstract"
#data_source = "hard and known"
#data_source = "nominalized"
data_source = "complete"

In [6]:
logging.set_verbosity_error()

In [7]:
MASK = "[MASK]"
#MASK = "<mask>"
KEYS = ["active", "passive", "nominalized"]

In [8]:
unmasker = PipelineCacheWrapper('fill-mask', model='bert-base-cased', top_k=10)
def model_to_tokens(sentence):
    result = unmasker(sentence)
    result = sorted(result, key=lambda entry: entry["score"], reverse=True)
    return [entry["token_str"] for entry in result]

In [9]:
get_hard_filter = create_hard_filter_getter(model_to_tokens, get_templates)
get_known_filter = create_known_filter_getter(model_to_tokens, get_templates, get_relation_cardinality)

In [10]:
TREx = LamaTRExData(relations = relations)
#TREx = AbstractTRExData(relations=relations)
#TREx = HardKnownTRExData(get_hard_filter, get_known_filter, relations = relations)

In [11]:
TREx.load()

In [12]:
#relations = ["P19", "P36", "P101", "P103","P106","P108", "P178", "P1001"]
#relations = ["P19", "P413", "P159", "P103"]
#relations = ["P138", "P136"]
#relations = ["P413"]
#relations = nominalized_relations
#relations = ['P37', 'P1412', 'P178', 'P103', 'P36', 'P407', 'P1376', 'P1001', 'P140', 'P463', 'P176'] # considered small
#considered_relations = ['P159', 'P37', 'P1412', 'P17', 'P178', 'P103', 'P36', 'P407', 'P276', 'P31', 'P364', 'P1376', 'P127', 'P279', 'P1001', 'P140', 'P463', 'P176'] #big
#relations = list(TREx.data.keys())
#relations = nominalized_relations
#relations = get_relations_with_last_digit(8)+get_relations_with_last_digit(9)

In [13]:
len(relations)

41

In [14]:
TOP_K = 1

In [15]:
#unmasker = pipeline('fill-mask', model='bert-large-uncased-whole-word-masking', top_k=100)
unmasker = PipelineCacheWrapper('fill-mask', model=model, top_k=10)

In [16]:
sum([len(val) for _, val in TREx.data.items()])

34039

In [17]:
#querier = TypologyQuerier(unmasker, relations, TOP_K, MASK, KEYS)
querier = TypologyQuerier(unmasker, relations, TOP_K, MASK)

In [18]:
querier.query(TREx.data)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 982/982 [00:48<00:00, 20.18it/s]


In [19]:
%%time
unmasker.save_to_cache()

CPU times: user 749 ms, sys: 67.2 ms, total: 817 ms
Wall time: 820 ms


In [20]:
querier.print_global_result()

total Result
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |       609 |   34039 |   1.78912  |
| compound         |       256 |   34039 |   0.752078 |
| complex          |       117 |   34039 |   0.343723 |
| compound-complex |        58 |   34039 |   0.170393 |
Total Result for 1:1
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |         0 |     937 |          0 |
| compound         |         0 |     937 |          0 |
| complex          |         0 |     937 |          0 |
| compound-complex |         0 |     937 |          0 |
Total Result for N:1
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |       457 |   20928 |   2.18368  |
| compound         |       189 |   20928 |   0.903096 |
| complex          |        96 |   20928 |   0.45

In [21]:
measure = "k@1 accuracy"
persister = ResultPersister(model=model, measure=measure, data_source=data_source)

In [22]:
persister.persist(querier.results_for_persistence())
persister.close()

In [23]:
TOP_K = 10

In [24]:
unmasker = PipelineCacheWrapper('fill-mask', model=model, top_k=TOP_K)

In [25]:
querier.reset()
querier.set_top_k(TOP_K)
querier.set_model(unmasker)

In [26]:
querier.query(TREx.data)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 982/982 [00:00<00:00, 32731.28it/s]


In [27]:
%%time
unmasker.save_to_cache()

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.1 µs


In [28]:
querier.print_global_result()

total Result
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |      1856 |   34039 |    5.45257 |
| compound         |      1326 |   34039 |    3.89553 |
| complex          |      1064 |   34039 |    3.12583 |
| compound-complex |      1348 |   34039 |    3.96016 |
Total Result for 1:1
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |         0 |     937 |          0 |
| compound         |         0 |     937 |          0 |
| complex          |         0 |     937 |          0 |
| compound-complex |         0 |     937 |          0 |
Total Result for N:1
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |      1237 |   20928 |    5.91074 |
| compound         |       739 |   20928 |    3.53115 |
| complex          |       590 |   20928 |    2.8

In [29]:
measure = "k@10 accuracy"
persister = ResultPersister(model=model, measure=measure, data_source=data_source)

In [30]:
persister.persist(querier.results_for_persistence())
persister.close()

In [31]:
TOP_K = 100

In [32]:
unmasker = PipelineCacheWrapper('fill-mask', model=model, top_k=TOP_K)

In [33]:
querier.reset()
querier.set_top_k(TOP_K)
querier.set_model(unmasker)

In [34]:
querier.query(TREx.data)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 982/982 [00:33<00:00, 29.72it/s]


In [35]:
%%time
unmasker.save_to_cache()

CPU times: user 9.25 s, sys: 620 ms, total: 9.87 s
Wall time: 10.1 s


In [36]:
querier.print_global_result()

total Result
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |      4253 |   34039 |   12.4945  |
| compound         |      3161 |   34039 |    9.28641 |
| complex          |      2754 |   34039 |    8.09072 |
| compound-complex |      3177 |   34039 |    9.33341 |
Total Result for 1:1
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |         0 |     937 |          0 |
| compound         |         0 |     937 |          0 |
| complex          |         0 |     937 |          0 |
| compound-complex |         0 |     937 |          0 |
Total Result for N:1
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |      2233 |   20928 |   10.6699  |
| compound         |      1497 |   20928 |    7.1531  |
| complex          |      1473 |   20928 |    7.0

In [37]:
measure = "k@100 accuracy"
persister = ResultPersister(model=model, measure=measure, data_source=data_source)

In [38]:
persister.persist(querier.results_for_persistence())
persister.close()